In [188]:

import re
def replace_entities(text, entity_mapping):
    # Sort entity mapping by length in descending order to handle overlapping matches
    sorted_mapping = sorted(entity_mapping, key=len, reverse=True)

    # Create a regular expression pattern to match entities at word boundaries
    pattern = r"\b(" + "|".join(re.escape(entity) for entity in sorted_mapping) + r")\b"

    # Replace matching entities in the text using the pattern
    replaced_text = re.sub(pattern, lambda match: entity_mapping[match.group()], text)

    replaced_text=replaced_text.replace(":","")
    return replaced_text
text = """
领导早上好[Sun][Sun][Sun][Sun]
世纪瑞丰矿产资源 今日报盘请查收：

1-江阴港：
[Coffee]PB粉 船名：鹏锦
铁61.37 硅3.70 铝2.49 磷0.100 硫0.020 水9.38

2-青岛港 
[Coffee]BRPL球团 船名：武士
FE:63.18,SI:3.81,AL:3.85,P:0.055,S:0.006,H2O:0.81

[Coffee]卡粉  船名：雨果
FE:64.97,SI:2.47,AL:1.20,P:0.052,Mn:0.195,H2O:8.70

盘面有波动，请以我公司最终确认为准！顺祝商祺！
"""

entity_mapping = {
    "FE": "铁",
    "SI": "硅",
    "AL": "铝",
    "P": "磷",
    "S": "硫",
    "H2O": "水",
    "Mn": "锰"
}

text = replace_entities(text, entity_mapping)
print(text)



领导早上好[Sun][Sun][Sun][Sun]
世纪瑞丰矿产资源 今日报盘请查收：

1-江阴港：
[Coffee]PB粉 船名：鹏锦
铁61.37 硅3.70 铝2.49 磷0.100 硫0.020 水9.38

2-青岛港 
[Coffee]BRPL球团 船名：武士
铁63.18,硅3.81,铝3.85,磷0.055,硫0.006,水0.81

[Coffee]卡粉  船名：雨果
铁64.97,硅2.47,铝1.20,磷0.052,锰0.195,水8.70

盘面有波动，请以我公司最终确认为准！顺祝商祺！



In [189]:
from pyhanlp import HanLP
import openpyxl

def extract_named_entities(text):


    segment = HanLP.newSegment().enableNameRecognize(True)
    term_list = segment.seg(text)
    named_entities = []
    for term in term_list:
        if term.nature.toString() == "nr"or "nt":
            named_entities.append(term.word)
    return named_entities



def extract_data(text, named_entities):
    ship_names = []
    commodities = []
    prices = []

    # Find ship names
    ship_name_match = re.findall(r"船名[：:](\S+)", text)
    if ship_name_match:
        ship_names.extend(ship_name_match)

    commodities_mapping = {"铁": "FE", "硅": "SI", "铝": "AL", "磷": "P", "硫": "S", "水": "H2O","锰": "Mn"}
    
    # Find commodities and prices
    for i in range(len(named_entities)):
        entity = named_entities[i]
        if entity in commodities_mapping:
            commodities.append(commodities_mapping[entity])
            # Look for the number after the commodity
            if i < len(named_entities) - 1:
                next_entity = named_entities[i + 1]
                if next_entity.replace('.', '', 1).isdigit():
                    prices.append(next_entity)
                else:
                    prices.append('')
    

    return ship_names, commodities, prices


def save_to_excel(ship_names, commodities, prices, filename):
    wb = openpyxl.Workbook()
    sheet = wb.active

    
    sheet['A1'] = "Ship Name"
    sheet['B1'] = "Commodity"
    sheet['C1'] = "Price"


    for i in range(len(commodities)):
        if i<6:
            sheet.cell(row=i+2, column=1).value = ship_names[0]
        elif i>=6 and i<12:
            sheet.cell(row=i+2, column=1).value = ship_names[1]
        elif i>=12 and i<18:
            sheet.cell(row=i+2, column=1).value = ship_names[2]
        sheet.cell(row=i+2, column=2).value = commodities[i] if i < len(commodities) else ""
        sheet.cell(row=i+2, column=3).value = prices[i] if i < len(prices) else ""

    wb.save(filename)



In [190]:
named_entities = extract_named_entities(text)
ship_names, commodities, prices = extract_data(text, named_entities)
filename = "extracted_data.xlsx"
save_to_excel(ship_names, commodities, prices, filename)
print(named_entities)
print("ship_names: ", ship_names)
print("commodities: ",commodities)
print("prices: ", prices)

['\n', '领导', '早上好', '[', 'Sun', '][', 'Sun', '][', 'Sun', '][', 'Sun', ']', '\n', '世纪', '瑞丰', '矿产资源', ' ', '今日', '报盘', '请', '查收', '：', '\n\n', '1', '-', '江阴', '港', '：', '\n', '[', 'Coffee', ']', 'PB', '粉', ' ', '船名', '：', '鹏', '锦', '\n', '铁', '61.37', ' ', '硅', '3.70', ' ', '铝', '2.49', ' ', '磷', '0.100', ' ', '硫', '0.020', ' ', '水', '9.38', '\n\n', '2', '-', '青岛港', ' \n', '[', 'Coffee', ']', 'BRPL', '球团', ' ', '船名', '：', '武士', '\n', '铁', '63.18', ',', '硅', '3.81', ',', '铝', '3.85', ',', '磷', '0.055', ',', '硫', '0.006', ',', '水', '0.81', '\n\n', '[', 'Coffee', ']', '卡', '粉', '  ', '船名', '：', '雨果', '\n', '铁', '64.97', ',', '硅', '2.47', ',', '铝', '1.20', ',', '磷', '0.052', ',', '锰', '0.195', ',', '水', '8.70', '\n\n', '盘面', '有', '波动', '，', '请', '以', '我', '公司', '最终', '确认', '为准', '！', '顺祝', '商', '祺', '！', '\n']
ship_names:  ['鹏锦', '武士', '雨果']
commodities:  ['FE', 'SI', 'AL', 'P', 'S', 'H2O', 'FE', 'SI', 'AL', 'P', 'S', 'H2O', 'FE', 'SI', 'AL', 'P', 'Mn', 'H2O']
prices:  ['61.37', '3.70', '2